In [22]:
import cv2  
import os
from PIL import Image
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score


In [23]:
INPUT_SIZE = 64
image_directory = r'E:/Project/2024 Project/BrainTumor_Lam/datasets/'

no_tumor_images = os.listdir(image_directory + 'sort_crop_no/')
yes_tumor_images = os.listdir(image_directory + 'sort_crop_yes/')
dataset = []
label = []

print(no_tumor_images)
print(yes_tumor_images)


['no0.jpg', 'no1.jpg', 'no10.jpg', 'no100.jpg', 'no1000.jpg', 'no1002.jpg', 'no1005.jpg', 'no1006.jpg', 'no1008.jpg', 'no101.jpg', 'no1010.jpg', 'no1012.jpg', 'no1013.jpg', 'no1014.jpg', 'no1015.jpg', 'no1016.jpg', 'no1017.jpg', 'no1018.jpg', 'no1019.jpg', 'no102.jpg', 'no1020.jpg', 'no1021.jpg', 'no1022.jpg', 'no1023.jpg', 'no1024.jpg', 'no1027.jpg', 'no1028.jpg', 'no1029.jpg', 'no103.jpg', 'no1030.jpg', 'no1032.jpg', 'no1033.jpg', 'no1034.jpg', 'no1035.jpg', 'no1036.jpg', 'no1037.jpg', 'no1038.jpg', 'no1039.jpg', 'no104.jpg', 'no1040.jpg', 'no1041.jpg', 'no1042.jpg', 'no1044.jpg', 'no1045.jpg', 'no1046.jpg', 'no1049.jpg', 'no105.jpg', 'no1051.jpg', 'no1053.jpg', 'no1054.jpg', 'no1055.jpg', 'no1057.jpg', 'no1058.jpg', 'no1059.jpg', 'no106.jpg', 'no1060.jpg', 'no1061.jpg', 'no1062.jpg', 'no1064.jpg', 'no1065.jpg', 'no1066.jpg', 'no1068.jpg', 'no1069.jpg', 'no107.jpg', 'no1070.jpg', 'no1071.jpg', 'no1072.jpg', 'no1073.jpg', 'no1074.jpg', 'no1076.jpg', 'no1078.jpg', 'no108.jpg', 'no1081.

In [24]:
# def crop_image(image):
#     if image is None:
#         print("Lỗi: Không thể đọc tệp ảnh. Hãy kiểm tra lại đường dẫn.")
#     else:
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#         # Sử dụng Canny để phát hiện cạnh
#         edges = cv2.Canny(blurred, 50, 150)

#         # Tìm các đường viền trong ảnh
#         contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#         # Chọn đường viền lớn nhất (giả sử vật thể lớn nhất là vật thể cần tìm)
#         contour = max(contours, key=cv2.contourArea)

#         # Vẽ viền lên ảnh gốc
#         cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)  # Màu viền là xanh lá cây

#         # Tìm tọa độ 4 điểm ngoài cùng
#         leftmost = tuple(contour[contour[:, :, 0].argmin()][0])
#         rightmost = tuple(contour[contour[:, :, 0].argmax()][0])
#         topmost = tuple(contour[contour[:, :, 1].argmin()][0])
#         bottommost = tuple(contour[contour[:, :, 1].argmax()][0])

#         # Xuất tọa độ 4 điểm
#         # print("Leftmost point:", leftmost)
#         # print("Rightmost point:", rightmost)
#         # print("Topmost point:", topmost)
#         # print("Bottommost point:", bottommost)

#         # Xác định tọa độ của hình vuông bao quanh 4 điểm ngoài cùng
#         x_min = min(leftmost[0], rightmost[0])
#         x_max = max(leftmost[0], rightmost[0])
#         y_min = min(topmost[1], bottommost[1])
#         y_max = max(topmost[1], bottommost[1])

#         # Chiều dài cạnh hình vuông (lấy giá trị lớn nhất giữa chiều rộng và chiều cao của vùng cần cắt)
#         square_size = max(x_max - x_min, y_max - y_min)

#         # Điều chỉnh tọa độ để đảm bảo hình vuông
#         x_center = (x_min + x_max) // 2
#         y_center = (y_min + y_max) // 2

#         x_min = x_center - square_size // 2
#         x_max = x_min + square_size
#         y_min = y_center - square_size // 2
#         y_max = y_min + square_size

#         # Đảm bảo tọa độ không vượt quá kích thước của ảnh
#         x_min = max(0, x_min)
#         y_min = max(0, y_min)
#         x_max = min(image.shape[1], x_max)
#         y_max = min(image.shape[0], y_max)

#         # Cắt ảnh để tạo ra ảnh hình vuông
#         cropped_image = image[y_min:y_max, x_min:x_max]

#         # Tạo ảnh mới với nền đen
#         new_size = square_size
#         new_image = np.zeros((new_size, new_size, 3), dtype=np.uint8)

#         # Tính toán vị trí để đặt hình vuông vào giữa ảnh nền đen
#         x_offset = (new_size - cropped_image.shape[1]) // 2
#         y_offset = (new_size - cropped_image.shape[0]) // 2

#         # Chèn ảnh đã cắt vào ảnh mới
#         new_image[y_offset:y_offset + cropped_image.shape[0], x_offset:x_offset + cropped_image.shape[1]] = cropped_image
#     return new_image


In [25]:

for i , image_name in enumerate(no_tumor_images) :
    if(image_name.split('.')[1] == 'jpg') :
        image=cv2.imread(image_directory+ 'sort_crop_no/' + image_name)
        # crop_image(image)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)


for i , image_name in enumerate(yes_tumor_images) :
    if(image_name.split('.')[1] == 'jpg') :
        image=cv2.imread(image_directory+ 'sort_crop_yes/' + image_name)
        # crop_image(image)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

print(dataset)
print(label)

[array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,

In [26]:
np.array(dataset)  
label = np.array(label)  

x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2 )        

x_train = np.array(x_train, dtype=float)
x_test= np.array(x_test, dtype=float)

x_train_normalized=x_train/255
x_test_normalized=x_test/255

# y_train=to_categorical(y_train, num_classes=2)
# y_test=to_categorical(y_test, num_classes=2)

print(x_train_normalized, x_test_normalized, y_train, y_test)

[[[[0.         0.         0.        ]
   [0.         0.         0.00392157]
   [0.         0.         0.        ]
   ...
   [0.00392157 0.00392157 0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.00392157 0.00392157 0.00392157]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  ...

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0. 

In [27]:
#Building the model
#64,64,3
model=Sequential()  

model.add(Conv2D(32, (3, 3),input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))      
model.add(Activation('relu'))                      
model.add(MaxPooling2D(pool_size=(2, 2)))         

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [28]:
# from tensorflow.keras.optimizers import Adam
# optimizer1 = Adam(learning_rate=0.003) 

In [29]:
# model.compile(loss='categorical_crossentropy', optimizer= optimizer1,  metrics=['accuracy'])
# import log
# file_logging_callback  = log.FileLoggingCallback()

# model.fit(x_train_normalized, y_train, batch_size=16, verbose=1, epochs=10, 
#           validation_data=(x_test_normalized,y_test), shuffle=False
#           ,callbacks=[file_logging_callback ])
# model.save('New_Braintumor10EpochsCategorical.h5')

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

optimizer1 = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer1,  
              metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

import log
file_logging_callback = log.FileLoggingCallback()

history = model.fit(
    x_train_normalized, 
    y_train, 
    batch_size=32, 
    epochs=10, 
    verbose=1, 
    validation_data=(x_test_normalized, y_test), 
    shuffle=True,
    callbacks=[file_logging_callback, reduce_lr, early_stopping]
)

# Lưu mô hình
save_path = 'E:/Project/2024 Project/BrainTumor_Lam/models/brain_tumor_tensorflow_v2.h5'
model.save(save_path)



Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5273 - loss: 0.7564 - val_accuracy: 0.5824 - val_loss: 0.6759 - learning_rate: 0.0010
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5948 - loss: 0.6651 - val_accuracy: 0.6961 - val_loss: 0.5732 - learning_rate: 0.0010
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7101 - loss: 0.5506 - val_accuracy: 0.7564 - val_loss: 0.4856 - learning_rate: 0.0010
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7810 - loss: 0.4592 - val_accuracy: 0.8190 - val_loss: 0.4230 - learning_rate: 0.0010
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8567 - loss: 0.3576 - val_accuracy: 0.8469 - val_loss: 0.3482 - learning_rate: 0.0010
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8902 - loss: 0.2710 - val_accuracy: 0.8886 - val_loss: 0.2908 - learning_rate: 0.0010
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9249 - loss: 0.2069 - val_acc

In [30]:
y_pred = model.predict(x_test_normalized)
y_pred_classes = (y_pred > 0.5).astype("int32").flatten()
# y_true should not be categorical, should match the output of y_pred_classes
f1 = f1_score(y_test, y_pred_classes)
print(f"F1 Score: {f1}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
F1 Score: 0.9106628242074928
